# Домашнее задание №2
## Знакомство с языком обработки данных SAS Base
### Циклы. Массивы. Retain. Опции. Сортировка.

Солонин Цветков Осипова

### 0. Подготовка файлов для выполнения ДЗ.
Вместе с данным файлом в архиве находятся данные для выполнения ДЗ: 
- 4 набора данных SAS: **order_fact.sas7bdat**, **order_summary.sas7bdat**,  **orders_midyear.sas7bdat**, **customer_dim.sas7bdat**

Скопируйте данные в директорию на вашем компьютере, которая связана с виртуальной машиной. Далее всю работу мы будем проводить в этой директории.


### 1. Создание библиотеки.
C помощью оператора libname создайте библиотеку SAS с именем *_Orion_*, которая "смотрит"/"является ссылкой" на директурию, в которую вы скопировали данные для выполнения задания.

In [2]:
LIBNAME Orion '/folders/myfolders/HW2';

### 2.1   Создание набегающей суммы 
Набор данных **orion.order_fact** содержит информацию о заказах за несколько лет, отсортированную по переменной `Order_Date`. Каждое наблюдение представляет собой один заказ, а переменная `Total_Retail_Price` содержит полную стоимость заказа. 

Требуется проанализировать рост продаж в компании Orion Star **за период** с 1 ноября 2008 по 14 декабря 2008.  
- Приведенный ниже код создаёт набор данных **work.mid_q4** из **orion.order_fact**.
- Измените программу, чтобы вычислить набегающий итог суммы заказов, сделанных за период, в переменную `Sales2Dte`. 
- Также создайте аккумулирующую переменную `Num_Orders`, содержащую число заказов на определённую дату с начала периода. Каждое наблюдение считается отдельным заказом. 

In [4]:
data work.mid_q4;
set orion.order_fact;
retain Sales2Dte 0;
if ('01NOV2008'd <= Order_Date <= '11DEC2008'd) then
	do;
	Sales2Dte = Sales2Dte + Total_Retail_Price;
	output;
	end;
run;


### 2.2 Агрегирование данных на шаге DATA 
Набор данных **orion.order_summary** содержит информацию о продажах за определённый год для каждого клиента. Информация разбита по месяцам. В какие-то месяцы клиент мог не размещать заказы. 

a.   Отсортируйте входной набор данных **orion.order_summary** по `Customer_ID`.  Используйте параметр _OUT=_, чтобы не перезаписывать исходный набор данных. Задайте для выходного набора данных название **work.sumsort**. 

In [6]:
proc sort data = orion.order_summary 
out = work.sumsort nodup;
by Customer_ID;
run;

b.   Создайте новый набор данных, содержащий общую стоимость заказов для каждого клиента. 
- Назовите новый набор данных **work.customers**. 
- Назовите новую переменную `Total_Sales`. Эта переменная должна содержать сумму стоимости заказов за все месяцы для каждого клиента. 

In [7]:
data work.customers;
    set orion.order_summary;
    by Customer_ID;
    retain Total_Sales 0;
    if first.Customer_ID then Total_Sales = 0;
    Total_Sales = Total_Sales + Sale_Amt;
    if last.Customer_ID then
    do;
        Total_Sales = Total_Sales + Sale_Amt;
        output work.customers;
        Total_Sales = 0;
        drop Order_Month Sale_Amt;
    end;
run;

### 2.3 Выполнение вычислений в циклах DO 
Отделу начисления заработной платы в Orion Star нужно запланировать суммарные затраты на сотрудников (включая заработную плату, пенсии, затраты на медицинское обслуживание) на несколько лет вперёд, учитывая предполагаемые коэффициенты по их увеличению. 


**a.**   Внесите следующие изменения в код, приведеный ниже: 
- Добавьте цикл **`DO`**, содержащий операторы для вычисления значений переменных `Wages`, `Retire` и `Medical`.  
- Используйте переменные `Start` и `Stop` как начальное и конечное значение индексной переменной `Year`.  Не сохраняйте эти переменные в выходной набор данных.
- Предположим, что значения коэффициентов для ежегодного увеличения переменных задаются следующей таблицей. Например, чтобы вычислить Wages, нужно использовать формулу: 
        wages = wages * 1.06; 
        
        Переменная   Текущее значение  Оценка ежегодного роста 
           Wages       $12,874,000        6.0% 
           Retire      1,765,000          1.4% 
           Medical     649,000            9.5%
- Создайте еще одну переменную, `Total_Cost`, равную сумме значений переменных `Wages`, `Retire` и `Medical` за определённый год. 
- Выведите по одному наблюдению на каждый год.  Результирующий набор данных должен содержать 10 наблюдений.  

In [8]:
data future_expenses;
   Wages=12874000;
   Retire=1765000;
   Medical=649000;
   start = year(input("&sysdate9",date9.))+1;
   stop = start+9;
   /* ... do loop ...*/
  
	do Year = start to stop;
	Wages = Wages * 1.06**(Year - start + 1);
	Retire = Retire * 1.014**(Year - start + 1);
	Medical = Medical * 1.095**(Year - start + 1);
	Total_Cost = Medical + Retire + Wages;
	output;
	drop Medical Retire Wages start stop;
	end;
run;

**б.**   Доход корпорации за прошлый год составил $50 000 000 (переменная `Income`). Рост дохода предполагается на уровне одного процента в год.  
    Измените программу выше (поулченную в пункте **2.3 a.**) так, чтобы цикл **`DO`** остановился, не ранее момента, **когда ежегодные расходы превысят ежегодный доход**. 

In [9]:
data future_expenses;
   Wages=12874000;
   Retire=1765000;
   Medical=649000;
   Income = 50000000;
   Total_Cost = Medical + Retire + Wages;
   Year_Start = year(input("&sysdate9",date9.))+1;
   Year = Year_Start;
   do until (Total_Cost > Income);
   	Wages = Wages * 1.06**(Year - Year_Start + 1);
	Retire = Retire * 1.014**(Year - Year_Start + 1);
	Medical = Medical * 1.095**(Year - Year_Start + 1);
	Total_Cost = Medical + Retire + Wages;
	Income = Income * 1.01**(Year - Year_Start + 1);
	Year = Year + 1;
	output;
	drop Medical Retire Wages Year_Start;
   end;
run;

### 2.4  Использование массива для _табличного поиска_
Менеджер отдела продаж хочет определить подходящих клиентов для будущей промо-акции. Используйте **orion.orders_midyear** и таблицу для поиска, чтобы создать новый набор данных **preferred_cust**.  

**a.**   Внесите следующие изменения в код, приведенный ниже:  
- Создайте временную таблицу для поиска (т.е. массив) `Target`, которая содержит целевые значения суммы покупок за каждый месяц:     
    200, 400, 300, 100, 100, 200 (_при необходимости обратитесь к документации, чтобы узнать синтаксис указания значений элементов массва при его объявлении_) 
-  Создайте новые переменные, `Over1`, `Over2`, …,`Over6`, которые содержат разницу между суммой покупок клиента в этом месяце и целевым значением для соответствующего месяца. 
- Примените цикл **`DO`** и вычислите значения переменных `Over1` ... `Over6`, только когда сумма покупок клиента в этом месяце превышает соответствующее целевое значение.
    **_Замечание:_** Если сумма покупок не превышает целевое значение, не выполняйте этот расчёт. 
- Сохраните сумму переменных `Over1` ... `Over6` в другой новой переменной, `Total_Over`. 

In [10]:
data preferred_cust;
   set orion.orders_midyear;
   array Mon{6} Month1-Month6;
   keep Customer_ID Over1-Over6 Total_Over;
   array Target{6} t1-t6 (200 400 300 100 100 200);
   array Over{6} (0 0 0 0 0 0);
   do i = 1 to 6;
   	if Mon{i} - Target{i} > 0 then Over{i} = Mon{i} - Target{i};
   	else Over{i} = ' ';
   end;
   Total_Over = sum(of Over[*]);
   output;
run;

### 2.5 Изучите пример использования оператора цикла DO с условием остановки 
Доход корпорации Orion за прошлый год составил  $ \$ 50 000 000 $ , а расходы - $ \$ 38 750 000 $ . Прогнозируемый рост дохода составляет 1% в год, а расходов - 2% в год.   

a.   Создайте набор данных с названием **work.expenses**, который содержит прогнозы ежегодных доходов и расходов.
- Для его создания испольузйте оператор **`DO`** с условием. 
- Остановите цикл, если расходы превысят доходы, или после 30 итераций (что произойдёт раньше).

In [11]:
data expenses; 
    Income= 50000000;  
    Expenses = 38750000;
    do Year=1 to 30 until (Expenses > Income);   
        income+(income * .01);   
        expenses+(expenses * .02);  
    end;
run; 

### 2.6 Определение экстремальных значений внутри групп данных 
Набор данных **orion.customer_dim** содержит информацию о клиентах Orion Star. Используйте обработку, основанную на переменных `First.`/`Last.` для создания показанного ниже отчёта. 

| Customer_Type |	oldest |	youngest|	o_ID|	y_ID|	agerange|
|---|---|---|---|---|---|
|Internet/Catalog Customers|	08JUL1938|	18AUG1973|	29|	54655|	35.1|
|Orion Club members high activity|	28SEP1938|	24OCT1990|	89|	46966|	52.1|
|Orion Club members medium activity|	20JAN1938|	16SEP1992|	70059|	2806|	54.7|
|Orion Club Gold members high activity|	16JAN1938|	25JUL1988|	50|	39|	50.5|
|Orion Club Gold members low activity|	19DEC1973|	21JUL1992|	70201|	13|	18.6|
|Orion Club Gold members medium activity|	16MAY1953|	09JUL1983|	215|	5|	30.1|
|Orion Club members low activity|	14APR1943|	18JUN1988|	70108|	75|	45.2|

Выведите информацию для самого пожилого и самого молодого клиента для каждого значения переменной `Customer_Type`.
- Переменная `o_ID` должна содержать значение `Customer_ID` для самого **пожилого** клиента, а `y_ID` - значение `Customer_ID` для самого **молодого** в каждой группе.   
- Создайте переменную `agerange`, которая вычисляется как разница между возрастами самого пожилого и самого молодого клиентов.  
- Для более точного вычисления этой переменной используйте переменную `Customer_BirthDate`, а не `Customer_Age`. 

**_Замечание:_** Существуют различные способы расчёта числа лет между двумя датами. Например, можно поделить число дней между этими датами на 365.25, или воспользоваться функцией _YRDIF_.

In [3]:
proc sort data=orion.customer_dim out=work.customers; 
by Customer_Type Customer_Age; 
run; 

data work.agecheck; 
set work.customers;
by Customer_Type;
if first.Customer_Type then
do;
retain youngest
y_ID;
youngest = Customer_BirthDate;
y_ID = Customer_ID;
format youngest date9.;
end;
if last.Customer_Type then
do;
oldest = Customer_BirthDate;
o_ID = Customer_ID;
agerange = round(YRDIF(oldest, youngest),0.1);
format oldest date9.;
output;
end;
keep Customer_Type youngest oldest o_ID y_ID agerange;
run; 

proc print data = work.agecheck; run;

Obs,Customer_Type,youngest,y_ID,oldest,o_ID,agerange
1,Internet/Catalog Customers,15APR1973,27,08JUL1938,29,34.8
2,Orion Club members high activity,24OCT1990,46966,28SEP1938,89,52.1
3,Orion Club members medium activity,16SEP1992,2806,20JAN1938,70059,54.7
4,Orion Club Gold members high activity,25JUL1988,39,16JAN1938,50,50.5
5,Orion Club Gold members low activity,21JUL1992,13,19DEC1973,70201,18.6
6,Orion Club Gold members medium activity,09JUL1983,5,16MAY1953,215,30.1
7,Orion Club members low activity,18JAN1988,34,14APR1943,70108,44.8
